## Heart Disease Classification – ML Assignment 2

# STUDENT INFORMATION

# BITS ID: 2025AA05584
# Name: PRAKARTI PANAGARIA

# Email: PRAKARTI PANAGARIA 2025aa05584@wilp.bits-pilani.ac.in

# Date: 12-02-2026

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Metrics
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    matthews_corrcoef,
    confusion_matrix
)
import time
import warnings
warnings.filterwarnings('ignore')
# Set random seed for reproducibility
np.random.seed(42)
print('✓ Libraries imported successfully')

✓ Libraries imported successfully


# Section 1: Dataset Selection and Loading
Requirements:
Implement multiple classification models
Minimum Feature Size: 12 Minimum Instance Size: 500
Public dataset (UCI/Kaggle)
binary classification problem or a multi-class classifi cation problem

In [2]:
from google.colab import files
uploaded = files.upload()

Saving heart_disease_cleaned.csv to heart_disease_cleaned (2).csv


In [3]:
data=pd.read_csv('heart_disease_cleaned.csv')
data.shape
data.head()
data.columns
data.info()





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        919 non-null    int64  
 1   age       919 non-null    int64  
 2   sex       919 non-null    object 
 3   dataset   919 non-null    object 
 4   cp        919 non-null    object 
 5   trestbps  919 non-null    float64
 6   chol      919 non-null    float64
 7   fbs       919 non-null    bool   
 8   restecg   919 non-null    object 
 9   thalch    919 non-null    float64
 10  exang     919 non-null    bool   
 11  oldpeak   919 non-null    float64
 12  slope     919 non-null    object 
 13  ca        919 non-null    float64
 14  thal      919 non-null    object 
 15  num       919 non-null    int64  
dtypes: bool(2), float64(5), int64(3), object(6)
memory usage: 102.4+ KB


In [4]:
obj_cols = data.select_dtypes(include='object').columns
obj_cols
for col in obj_cols:
    print(col, data[col].unique())

list(data.columns)

sex ['Male' 'Female']
dataset ['Cleveland' 'Hungary' 'Switzerland' 'VA Long Beach']
cp ['typical angina' 'asymptomatic' 'non-anginal' 'atypical angina']
restecg ['lv hypertrophy' 'normal' 'st-t abnormality']
slope ['downsloping' 'flat' 'upsloping']
thal ['fixed defect' 'normal' 'reversable defect']


['id',
 'age',
 'sex',
 'dataset',
 'cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalch',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal',
 'num']

In [5]:
    # Encoding
    data['sex'] = data['sex'].map({'Male': 1, 'Female': 0})
    data = pd.get_dummies(
    data,
    columns=['dataset', 'cp', 'restecg', 'slope', 'thal'],
    drop_first=True
    )
    bool_cols = data.select_dtypes(include='bool').columns
    data[bool_cols] = data[bool_cols].astype(int)
    data.dtypes
    data.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        919 non-null    int64  
 1   age                       919 non-null    int64  
 2   sex                       919 non-null    int64  
 3   trestbps                  919 non-null    float64
 4   chol                      919 non-null    float64
 5   fbs                       919 non-null    int64  
 6   thalch                    919 non-null    float64
 7   exang                     919 non-null    int64  
 8   oldpeak                   919 non-null    float64
 9   ca                        919 non-null    float64
 10  num                       919 non-null    int64  
 11  dataset_Hungary           919 non-null    int64  
 12  dataset_Switzerland       919 non-null    int64  
 13  dataset_VA Long Beach     919 non-null    int64  
 14  cp_atypica

The original multi-class target variable was converted into a binary target where 0 indicates absence of heart disease and 1 indicates presence of heart disease. The original target column was removed to prevent data leakage.

In [6]:
data['target'] = data['num'].apply(lambda x: 1 if x >= 1 else 0)
data.drop(columns=['num'], inplace=True)
data['target'].value_counts()
X = data.drop(columns=['target'])
y = data['target']


Section 2: Train-test split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)
y_train.value_counts(normalize=True)
y_test.value_counts(normalize=True)


,proportion
target,
1,0.554348
0,0.445652


Feature Scaling

In [8]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Logistic regression Model 1
Logistic regression is used for binary classification, where the model predicts probabilities using the sigmoid function and is trained using log loss. Model performance is evaluated using metrics such as accuracy, precision, recall, F1 score, and AUC.

In [9]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_scaled, y_train)
y_pred = lr.predict(X_test_scaled)
y_prob = lr.predict_proba(X_test_scaled)[:, 1]


In [10]:
# Evaluation Metrics
lr_accuracy = accuracy_score(y_test, y_pred)
lr_precision = precision_score(y_test, y_pred)
lr_recall = recall_score(y_test, y_pred)
lr_f1 = f1_score(y_test, y_pred)
lr_auc = roc_auc_score(y_test, y_prob)
lr_mcc = matthews_corrcoef(y_test, y_pred)

lr_accuracy, lr_precision, lr_recall, lr_f1, lr_auc, lr_mcc

(0.9239130434782609,
 0.9230769230769231,
 0.9411764705882353,
 0.9320388349514563,
 np.float64(0.9503825920612147),
 np.float64(0.8458432675391844))

In [11]:
cm = confusion_matrix(y_test, y_pred)
cm


array([[74,  8],
       [ 6, 96]])

# Decision Tree (MODEL 2)

In [12]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
y_prob_dt = dt.predict_proba(X_test)[:, 1]


In [13]:
dt_accuracy = accuracy_score(y_test, y_pred_dt)
dt_precision = precision_score(y_test, y_pred_dt)
dt_recall = recall_score(y_test, y_pred_dt)
dt_f1 = f1_score(y_test, y_pred_dt)
dt_auc = roc_auc_score(y_test, y_prob_dt)
dt_mcc = matthews_corrcoef(y_test, y_pred_dt)

dt_accuracy, dt_precision, dt_recall, dt_f1, dt_auc, dt_mcc

(0.8804347826086957,
 0.8703703703703703,
 0.9215686274509803,
 0.8952380952380953,
 np.float64(0.8754184600669535),
 np.float64(0.7579384823238913))

In [14]:
confusion_matrix(y_test, y_pred_dt)

array([[68, 14],
       [ 8, 94]])

Decision Tree showed strong recall but slightly lower overall performance compared to Logistic Regression, indicating possible overfitting.

# MODEL 3: K-Nearest Neighbors (KNN)

In [15]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
y_pred_knn = knn.predict(X_test_scaled)
y_prob_knn = knn.predict_proba(X_test_scaled)[:, 1]

In [16]:
knn_accuracy = accuracy_score(y_test, y_pred_knn)
knn_precision = precision_score(y_test, y_pred_knn)
knn_recall = recall_score(y_test, y_pred_knn)
knn_f1 = f1_score(y_test, y_pred_knn)
knn_auc = roc_auc_score(y_test, y_prob_knn)
knn_mcc = matthews_corrcoef(y_test, y_pred_knn)

knn_accuracy, knn_precision, knn_recall, knn_f1, knn_auc, knn_mcc

(0.875,
 0.883495145631068,
 0.8921568627450981,
 0.8878048780487805,
 np.float64(0.9347202295552367),
 np.float64(0.7467534479979184))

In [17]:
confusion_matrix(y_test, y_pred_knn)

array([[70, 12],
       [11, 91]])

# MODEL 4: Naive Bayes

In [18]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
y_prob_nb = nb.predict_proba(X_test)[:, 1]


In [19]:
nb_accuracy = accuracy_score(y_test, y_pred_nb)
nb_precision = precision_score(y_test, y_pred_nb)
nb_recall = recall_score(y_test, y_pred_nb)
nb_f1 = f1_score(y_test, y_pred_nb)
nb_auc = roc_auc_score(y_test, y_prob_nb)
nb_mcc = matthews_corrcoef(y_test, y_pred_nb)

nb_accuracy, nb_precision, nb_recall, nb_f1, nb_auc, nb_mcc


(0.8641304347826086,
 0.8811881188118812,
 0.8725490196078431,
 0.8768472906403941,
 np.float64(0.9068627450980392),
 np.float64(0.7253841176466627))

In [20]:
confusion_matrix(y_test, y_pred_nb)


array([[70, 12],
       [13, 89]])

# MODEL 5: Random Forest (Ensemble)

In [21]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]


In [22]:
rf_accuracy = accuracy_score(y_test, y_pred_rf)
rf_precision = precision_score(y_test, y_pred_rf)
rf_recall = recall_score(y_test, y_pred_rf)
rf_f1 = f1_score(y_test, y_pred_rf)
rf_auc = roc_auc_score(y_test, y_prob_rf)
rf_mcc = matthews_corrcoef(y_test, y_pred_rf)

rf_accuracy, rf_precision, rf_recall, rf_f1, rf_auc, rf_mcc


(0.8967391304347826,
 0.8878504672897196,
 0.9313725490196079,
 0.9090909090909091,
 np.float64(0.9675394548063128),
 np.float64(0.7909638132453386))

In [23]:
confusion_matrix(y_test, y_pred_rf)

array([[70, 12],
       [ 7, 95]])

# MODEL 6: XGBoost (Ensemble)

In [24]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42,
    eval_metric="logloss",
    use_label_encoder=False
)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
y_prob_xgb = xgb.predict_proba(X_test)[:, 1]


In [25]:
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
xgb_precision = precision_score(y_test, y_pred_xgb)
xgb_recall = recall_score(y_test, y_pred_xgb)
xgb_f1 = f1_score(y_test, y_pred_xgb)
xgb_auc = roc_auc_score(y_test, y_prob_xgb)
xgb_mcc = matthews_corrcoef(y_test, y_pred_xgb)

xgb_accuracy, xgb_precision, xgb_recall, xgb_f1, xgb_auc, xgb_mcc


(0.9184782608695652,
 0.9142857142857143,
 0.9411764705882353,
 0.927536231884058,
 np.float64(0.9643711142993783),
 np.float64(0.8348711987539926))

In [26]:
confusion_matrix(y_test, y_pred_xgb)


array([[73,  9],
       [ 6, 96]])

# FINAL COMPARISON TABLE

In [27]:
import pandas as pd

results = pd.DataFrame({
    "Accuracy": [
        lr_accuracy, dt_accuracy, knn_accuracy,
        nb_accuracy, rf_accuracy, xgb_accuracy
    ],
    "AUC": [
        lr_auc, dt_auc, knn_auc,
        nb_auc, rf_auc, xgb_auc
    ],
    "Precision": [
        lr_precision, dt_precision, knn_precision,
        nb_precision, rf_precision, xgb_precision
    ],
    "Recall": [
        lr_recall, dt_recall, knn_recall,
        nb_recall, rf_recall, xgb_recall
    ],
    "F1 Score": [
        lr_f1, dt_f1, knn_f1,
        nb_f1, rf_f1, xgb_f1
    ],
    "MCC": [
        lr_mcc, dt_mcc, knn_mcc,
        nb_mcc, rf_mcc, xgb_mcc
    ]
}, index=[
    "Logistic Regression",
    "Decision Tree",
    "KNN",
    "Naive Bayes",
    "Random Forest",
    "XGBoost"
])

results


,Accuracy,AUC,Precision,Recall,F1 Score,MCC
Logistic Regression,0.923913,0.950383,0.923077,0.941176,0.932039,0.845843
Decision Tree,0.880435,0.875418,0.870370,0.921569,0.895238,0.757938
KNN,0.875000,0.934720,0.883495,0.892157,0.887805,0.746753
Naive Bayes,0.864130,0.906863,0.881188,0.872549,0.876847,0.725384
Random Forest,0.896739,0.967539,0.887850,0.931373,0.909091,0.790964
XGBoost,0.918478,0.964371,0.914286,0.941176,0.927536,0.834871


## Model Observations

| ML Model            | Observation about model performance                                                              |
| ------------------- | ------------------------------------------------------------------------------------------------ |
| Logistic Regression | Provided strong overall performance with high recall and AUC. Balanced and interpretable model.  |
| Decision Tree       | Good recall but lower overall stability. Shows signs of overfitting compared to ensemble models. |
| KNN                 | Moderate performance. Sensitive to feature scaling and dataset size.                             |
| Naive Bayes         | Naive Bayes showed comparatively lower performance than other models                                 |
| Random Forest       | Strong ensemble performance with improved stability and higher AUC.                              |
| XGBoost             | High performance with strong recall and AUC. Boosting improves classification robustness.        |
